In [82]:
import pandas as pd
import sqlite3
import math



In [2]:

conn = sqlite3.connect('D:\CSGOProGames\CSPAdb.sqlite3')
matches = pd.read_sql_query("""SELECT * FROM analyzer_MatchInfo""", conn)

playerdata = pd.read_sql_query("""SELECT *
                            FROM analyzer_PlayerData
                            JOIN analyzer_GameInfo 
                            ON analyzer_PlayerData.game_id = analyzer_GameInfo.id
                            WHERE played_map = 'inferno'
                            LIMIT 15000""", conn)

# Nades have different time stamps
nades = pd.read_sql_query("""SELECT * 
                        FROM analyzer_Nades
                        JOIN analyzer_GameInfo 
                        ON analyzer_Nades.game_id = analyzer_GameInfo.id
                        WHERE played_map = 'inferno'
                        LIMIT 15000""", conn)
conn.close()

In [25]:
fixNames = playerdata['team'].replace('Terrorist', 'T').replace('CounterTerrorist', 'CT')

In [28]:
playerdata['team'] = fixNames

In [29]:
playerdata.head()

,id,time,team,playernum,XPos,YPos,ZPos,weapon,game_id,id,played_map,t_score,ct_score,win_reason,match_id
0,376845,0.0,T,4,-1506.850,671.6946,-43.96875,Knife,217,217,inferno,0,0,TerroristWin,6
1,376846,0.0,T,5,-1655.827,418.4227,-63.34569,Knife,217,217,inferno,0,0,TerroristWin,6
2,376847,0.0,CT,6,2292.060,2027.6900,128.03130,Knife,217,217,inferno,0,0,TerroristWin,6
3,376848,0.0,CT,7,2363.130,1985.6600,128.03130,Knife,217,217,inferno,0,0,TerroristWin,6
4,376849,0.0,T,8,-1661.562,290.4704,-63.23479,Knife,217,217,inferno,0,0,TerroristWin,6


In [4]:
nades.head()

,id,time,XPos,YPos,ZPos,nade_type,game_id,id,played_map,t_score,ct_score,win_reason,match_id
0,3706,139.6,1192.6490,462.38760,125.80630,HE Nade,217,217,inferno,0,0,TerroristWin,6
1,3707,139.9,748.4474,-125.77450,631.44970,Flash Nade,217,217,inferno,0,0,TerroristWin,6
2,3708,141.6,786.7012,-83.28719,642.32220,Flash Nade,217,217,inferno,0,0,TerroristWin,6
3,3709,142.1,400.3172,2059.36300,130.88620,Decoy Nade,217,217,inferno,0,0,TerroristWin,6
4,3710,145.6,862.3829,-209.94330,97.66189,Decoy Nade,217,217,inferno,0,0,TerroristWin,6


In [5]:
currenttime = -1
    if row['time'] != currenttime:  # This is a new tick
        deadCT =  
        
    

SyntaxError: unexpected EOF while parsing (<ipython-input-5-7974daba3c16>, line 2)

In [9]:
bytick = playerdata.groupby(['game_id','time'])

In [92]:
players = [['CT', 'Dead'],['T', 'AK', 112, 114],['T', 'Dead']]

similarities = []

wantdeadT=0
wantdeadCT=0


for tick in bytick:
    print(tick)

    deadT = 5
    deadCT = 5
    tick = tick[1]
    for i,row in tick.iterrows():
        tempscore = []

        if row['team'] == 'T':
            deadT -= 1
        elif row['team'] == 'CT':
            deadCT -= 1
            
        for player in players:
            score = 0.0          
            if player[0] == 'T' and player[1] == 'Dead':
                tempscore.append('DT')
                continue
            elif player[0] == 'CT' and player[1] == 'Dead':
                tempscore.append('DCT')
                continue

            if row['team'] == player[0]:
                if row[8] == player[1]:
                    score += 1
                # This will need to be customized for each map
                distance = math.sqrt(((row['XPos']+1760)-(player[2]*9.75))**2 + ((row['YPos']+790)-(player[3]*9.75))**2)
                # Will need some kind of normalization
                score += 1/distance

                tempscore.append(score)
            else:
                tempscore.append(0)

        similarities.append(tempscore)
    
    deadT = 5
    deadCT = 5
    for i,row in enumerate(similarities):
        for j,column in enumerate(row):
            print(column)
            if column == 'DCT':
                if deadCT > 0:
                    similarities[i][j] = 2 # Determine good value
                else:
                    similarities[i][j] = 0
            elif column == 'DT':
                if deadT > 0:
                    similarities[i][j] = 2 # Determine good value
                else:
                    similarities[i][j] = 0            
            
    print(similarities)
    break

            
                

((217, 0.0),        id  time team  playernum      XPos       YPos       ZPos weapon  \
0  376845   0.0    T          4 -1506.850   671.6946  -43.96875  Knife   
1  376846   0.0    T          5 -1655.827   418.4227  -63.34569  Knife   
2  376847   0.0   CT          6  2292.060  2027.6900  128.03130  Knife   
3  376848   0.0   CT          7  2363.130  1985.6600  128.03130  Knife   
4  376849   0.0    T          8 -1661.562   290.4704  -63.23479  Knife   
5  376850   0.0   CT          9  2374.470  2171.8500  128.03130  Knife   
6  376851   0.0   CT         10  2433.620  2157.4100  128.03130  Knife   
7  376852   0.0   CT         11  2449.140  2010.2200  128.03130  Knife   
8  376853   0.0    T         12 -1491.000   490.0000  -63.96875  Knife   

   game_id   id played_map  t_score  ct_score    win_reason  match_id  
0      217  217    inferno        0         0  TerroristWin         6  
1      217  217    inferno        0         0  TerroristWin         6  
2      217  217    inferno    